In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


In [ ]:
import os
import numpy as np
from sklearn.metrics import accuracy_score
import torch
import torch.nn as nn
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

class XOR(nn.Module):

  def __init__(self, config):
    super(XOR, self).__init__()

    # 입력층 노드 수
    self.inode = config["input_node"]
    # 은닉층 데이터 크기
    self.hnode = config["hidden_node"]
    # 출력층 노드 수: 분류해야 하는 레이블 수
    self.onode = config["output_node"]

    # 활성화 함수로 Sigmoid 사용
    self.activation = nn.Sigmoid()

    # 신경망 설계
    self.linear1 = nn.Linear(self.inode, self.hnode, bias=True)
    self.linear2 = nn.Linear(self.hnode, self.onode, bias=True)  

  def forward(self, input_features):

    output1 = self.linear1(input_features)
    hypothesis1 = self.activation(output1)

    output2 = self.linear2(hypothesis1)
    hypothesis2 = self.activation(output2)

    return hypothesis2

In [ ]:
def load_dataset(file):
  data = np.loadtxt(file)
  print("DATA=",data)
  
  input_features = data[:,0:-1]
  print("X=",input_features)
  
  labels = np.reshape(data[:,-1],(4,1))
  print("Y=",labels)

 
  input_features = torch.tensor(input_features, dtype=torch.float)
  labels = torch.tensor(labels, dtype=torch.float)

  return (input_features, labels)

In [ ]:
def train(config):

  model = XOR(config).cuda()

  (input_features, labels) = load_dataset(config["input_data"])

  train_features = TensorDataset(input_features,labels)
  train_dataloader = DataLoader(train_features,shuffle=True,batch_size=config["batch_size"])

  loss_func = nn.BCELoss()

  optimizer = torch.optim.SGD(model.parameters(),lr=config["learn_rate"])


  for epoch in range(config["epoch"]+1):

    model.train()

    costs = []

    for(step,batch) in enumerate(train_dataloader) :

      batch = tuple(t.cuda() for t in batch)

      input_features,labels = batch

      optimizer.zero_grad()

      hypothesis = model(input_features)
      
      cost = loss_func(hypothesis,labels)
      cost.backward()
      optimizer.step()
      costs.append(cost.data.item())

      if epoch % 100 == 0:
        print("Average Loss = {0:f}".format(np.mean(cost)))
        torch.save(model.state.dict(),os.path.join(config["output_dir"],"epoch_{0:d}.pt".format(epoch)))
        do_test(model,train_dataloader)




In [ ]:
def test(config):
  model = XOR(config).cuda()

  model.load_state_dict(torch.load(os.path.join(config["output_dir"], config["model_name"])))

  (features, labels) = load_dataset(config)

  test_features = TensorDataset(features,labels)
  test_dataloader = DataLoader(test_features,shuffle=true,batch_size=config["batch_size"])

  do_test(model,test_dataloader)


  

In [ ]:
def tensor2list(input_tensor):
    return input_tensor.cpu().detach().numpy().tolist()

In [ ]:
def do_test(model, test_dataloader):

  # 평가 모드 셋팅
  model.eval()

  # Batch 별로 예측값과 정답을 저장할 리스트 초기화
  predicts, golds = [], []
  
  with torch.no_grad():

    for step, batch in enumerate(test_dataloader):
  
      # .cuda()를 통해 메모리에 업로드
      batch = tuple(t.cuda() for t in batch)

      input_features, labels = batch
      hypothesis = model(input_features)
     

      # ont-hot 표현으로 변경
      logits = (hypothesis>0.5).float()
      x = tensor2list(logits)
      y = tensor2list(labels)

      # 예측값과 정답을 리스트에 추가
      predicts.extend(x)
      golds.extend(y)
    
    print("PRED=",predicts)
    print("GOLD=",golds)
    print("Accuracy= {0:f}\n".format(accuracy_score(golds, predicts)))


In [ ]:
if(__name__=="__main__"):

    root_dir = "/gdrive/My Drive/colab/ann/xor"
    output_dir = os.path.join(root_dir, "output")
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    input_data ="{0:s}/{1:s}".format(root_dir,"train.txt")

    config = {"mode": "train",
              "model_name":"epoch_{0:d}.pt".format(1000),
              "output_dir":output_dir,
              "input_data" :input_data,
              "input_node":2,
              "hidden_node":10,
              "output_node":1,
              "learn_rate":1,
              "batch_size":4,
              "epoch":1000,
              }

    if(config["mode"] == "train"):
        train(config)
    else:
        test(config)

DATA= [[0. 0. 0.]
 [0. 1. 1.]
 [1. 0. 1.]
 [1. 1. 0.]]
X= [[0. 0.]
 [0. 1.]
 [1. 0.]
 [1. 1.]]
Y= [[0.]
 [1.]
 [1.]
 [0.]]


TypeError: ignored